In [15]:
import pandas as pd
import numpy as np
import random

from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

import re
from nltk import pos_tag
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer 

import seaborn as sns
import matplotlib.pyplot as plt


In [10]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

In [33]:
df = pd.read_csv('../../data/judge-1377884607_tweet_product_company.csv', encoding = 'latin1')
df.columns = ['text', 'product', 'emotion']
df = df[df['emotion'] != 'I can\'t tell']
df['product'].fillna('No Product', inplace = True)
df.dropna(inplace = True)

In [34]:
y = pd.get_dummies(df['emotion'])

In [35]:
y.head()

,Negative emotion,No emotion toward brand or product,Positive emotion
0,1,0,0
1,0,0,1
2,0,0,1
3,1,0,0
4,0,0,1


In [36]:
text_list = [str(x) for x in list(df['text'])]

corpus = []
for tweet in text_list:
    corpus.extend(tweet.split(' '))
    
len(corpus)

160187

tokenize tweets

In [20]:
text_list = [str(x) for x in list(df['text'])]
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(text_list)
list_tokenized_text = tokenizer.texts_to_sequences(text_list)
X_t = sequence.pad_sequences(list_tokenized_text, maxlen=100)

construct CNN with embedding layer

In [31]:
model = Sequential()

embedding_size = 128
model.add(Embedding(20000, embedding_size))
model.add(LSTM(25, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

fit model on tokenized data

In [32]:
model.fit(X_t, y, epochs=3, batch_size=32, validation_split=0.1)

Epoch 1/3
256/256 [==============================] - 14s 54ms/step - loss: 1.0069 - accuracy: 0.5636 - val_loss: 0.8923 - val_accuracy: 0.5945
Epoch 2/3
256/256 [==============================] - 13s 52ms/step - loss: 0.8881 - accuracy: 0.6060 - val_loss: 0.8305 - val_accuracy: 0.6374
Epoch 3/3
256/256 [==============================] - 14s 53ms/step - loss: 0.7695 - accuracy: 0.6824 - val_loss: 0.8149 - val_accuracy: 0.6275
